Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 4.25 ms, sys: 698 µs, total: 4.95 ms
Wall time: 11.3 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.129/10325/1 Dashboard: http://10.20.0.129:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 234 ms, sys: 31.8 ms, total: 266 ms
Wall time: 2.2 s


[1.821281928992832,
 1.0605488921073145,
 1.388249806038536,
 1.1998279078460379,
 1.2587696924289995,
 1.601176759237098,
 1.2613979612217951,
 1.5075003618863447,
 1.9978068400459943,
 1.9819419447049667,
 1.2799355014699674,
 1.16289910592522,
 1.6062171113395545,
 1.8393301746969175,
 1.4360528601892042,
 1.980384103782758,
 1.0641000178417865,
 1.8854054023867044,
 1.23266664627366,
 1.944834820105326,
 1.1877776113324403,
 1.9062694098554267,
 1.9714423401361438,
 1.6221046232036338,
 1.4350759113646996,
 1.124925597857642,
 1.2903626721222365,
 1.3700311956009128,
 1.3726516176010597,
 1.6211925922285242,
 1.4352638060816103,
 1.5389150800344136,
 1.829639583815427,
 1.1118574025950592,
 1.1191303461046769,
 1.9530543870061745,
 1.7691829630083111,
 1.9202099776922985,
 1.9911337269474267,
 1.122058299426397,
 1.5000344348971368,
 1.4991101798355069,
 1.8660130979981902,
 1.1163996340777214,
 1.4935213347595908,
 1.219297377435772,
 1.3675643103616917,
 1.5415121285628537,
 1.24

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.1333638724672719,
 1.015303590834186,
 1.0431034228038811,
 1.2652050254821514,
 1.684125005156146,
 1.187674019704495,
 1.9606705466667524,
 1.6427444000507765,
 1.8752651663802715,
 1.752807338316213,
 1.1360194273390223,
 1.5282155698518254,
 1.2819730319185068,
 1.082944254240286,
 1.9753811964935961,
 1.0893379505086882,
 1.958386493832009,
 1.8870351293225867,
 1.960400673438616,
 1.4585462668763691,
 1.5878824269881822,
 1.5867675147869749,
 1.5605268882531842,
 1.3650228855098985,
 1.1137754650541087,
 1.9413704599094883,
 1.7580226833490382,
 1.3840774950982278,
 1.872287965481444,
 1.804795424538766,
 1.0351814541448383,
 1.725784003664212,
 1.1634049105709376,
 1.361418917326167,
 1.0446206393466655,
 1.2758258036594223,
 1.463258422553275,
 1.8494814066392788,
 1.280163763370651,
 1.7124928850373795,
 1.4041954359074453,
 1.754664712313323,
 1.734300779225768,
 1.6476548996901412,
 1.8865557292631507,
 1.3498215294085134,
 1.8351745829734953,
 1.2305703487657529,
 1.6725

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 185 ms, sys: 11.4 ms, total: 196 ms
Wall time: 2.45 s
